# Multi document text summarization using Lexrank

### Authors: 
1. Pragnesh Krishnan Ramachandran Vidyasagar
2. Wei Wang

In [1]:
import math
from collections import Counter, defaultdict
import re
import numpy as np
from scipy.sparse import csr_matrix
from nltk import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import os
class LexRank:
    # docs: list of all documents, canonicalized
    # S: list of all sentences, canonicalized
    # S_orig: list of all sentences as they originally appeared
    # idf_mod_cosines: All idf modified cosines, 2d array
    # tf_matrix: Term frequency matrix for all sentences - rows are sentences, 
    # cols are words
    # word_index_dict: Dictionary of word to tf_matrix column index
    # idfs: All idfs saved, same indexing as tf_matrix columns
    # cluster_name: name of the cluster being summarized
    
    # Method: init
    # Inputs: dir_path - Directory where the cluster of files for summarization
    #         are stored
    #         task : default 2 - DUC task 2 vs DUC task 4
    #               Task 2 of both DUC 2003 and 2004 involve generic summarization 
    #               of news documents clusters. 
    #               Task 4 First set (Task 4a) is composed of Arabic-to-English machine 
    #               translations of 24 news clusters. Second set (Task 4b) is the human 
    #               translations of the same clusters. All data sets are in English. 
    # Outputs: None
    # Description: Initialization routine!
    #              If the input of task 1 or 2 is passed, documents from the folder
    #              are read and stored in docs list. 
    #              Tokenize documents in docs into sentences and store them in self.docs!
    #              Each sentence is converted to lower case and accumulated in self.S
    #              TF-IDF is calculated next for all words. Because IDF is at document level
    #              and not at sentence level, the TfIdfVectorizer is not used.
    #              
    def __init__(self, dir_path, task=2):
        self.docs = []
        self.S = []
        self.S_orig = []
        self.word_index_dict = defaultdict()
        self.sentence_index_dict = defaultdict()
        
        if not(dir_path.endswith('/')):
            dir_path += '/'
        
        self.cluster_name = dir_path.split('/')[-2]
        #print(self.cluster_name,"\n")
        
        docs = []
        if task == 1 or task == 2:
            for f in sorted(os.listdir(dir_path)):
                if f.endswith('.0'):
                    continue
                unfiltered_doc = open(dir_path + f).read()
                relevant_match = re.search(r"<TEXT>\n([\w\W]*)\n<\/TEXT>", unfiltered_doc)
                if relevant_match is None:
                    print("in file:", f, " Relevant match: None \n ", unfiltered_doc)
                docs.append(re.sub(r"\((end )?(italics|bold|underline)\)","",relevant_match.group(1).replace('\n','')))
        elif task == 3 or task == 4:
            for f in sorted(os.listdir(dir_path)):
                if f.endswith('0'):
                    continue
                unfiltered_doc = '\n'.join(list(open(dir_path + f))[1:])
                relevant_match = re.search(r"\(.*\) ?-? ?([\w\W]*)", unfiltered_doc)
                docs.append(re.sub(r"\((end )?(italics|bold|underline)\)","",relevant_match.group(1)))
        else:
            raise Exception('task can only be 1, 2, 3, or 4')
        
        for doc in docs:
            sentences = list(filter(lambda x: len(x) > 1, sent_tokenize(doc)))
            lower_sentences = list(filter(None, 
                                          map(
                lambda x: ' '.join(
                re.findall(
                r"((?:(?:[a-z0-9]\.){2,}|[a-z0-9]+[-']?[a-z0-9]+|\d*[-,'.]?[\da-z0-9]+|[a-z0-9]+)+)", 
                    x.lower())), sentences)))
            self.S += lower_sentences
            self.S_orig += sentences
            self.docs.append(' '.join(lower_sentences))
            
        # Compute Tf-IDF
        # Set up term frequency matrix things for all sentences across documents
        vectorizer = CountVectorizer(token_pattern='(?u)\\S+')
        self.tf_matrix = vectorizer.fit_transform(self.S)
        self.word_index_dict = {key: value for value,key in enumerate(vectorizer.get_feature_names())}
        
        # IDF things
        self.idfs            = self.compute_idfs(vectorizer)
        self.idf_mod_cosines = self.compute_idf_modified_cosines()

    # Method: compute_idfs
    # Inputs: Vectorizer: 
    #                Count vectorizer are stored
    # Outputs: idfs: output of method compute_idfs_document
    # Description:   Invoke IDF computation at document level and not at
    #                sentence level!
    def compute_idfs(self, vectorizer):
        return self.compute_idfs_document(vectorizer)
#         return self.compute_idfs_sentence(vectorizer)
        
    # Method: compute_idfs_document
    # Inputs: Vectorizer: 
    #               Count vectorizer are stored
    # Outputs: idfs:output of idfs across documents
    # Description:  an X matrix is built, where the columns are all words
    #               in all documents. Each document corresponds to a row
    #               N - No of documents in the cluster
    #               ni - no of documents in which the word is present
    #               Return log(n/ni)
    def compute_idfs_document(self, vectorizer):
        # DOCUMENT idf
        X = vectorizer.fit_transform(self.docs)
        N = len(self.docs)
        words = vectorizer.get_feature_names()
        idfs = np.zeros(len(words))
        for word in words:
            index = words.index(word)
            ni = X.getcol(index).count_nonzero()
            idfs[self.word_index_dict[word]] = np.log10(N/ni)
        return idfs
    
#    def compute_idfs_sentence(self, vectorizer):
#        # SENTENCE idf
#        N = len(self.S)
#        words = vectorizer.get_feature_names()
#        idfs = np.zeros(len(words))
#        for word in words:
#            index = words.index(word)
#            ni = self.tf_matrix.getcol(index).count_nonzero()
#            idfs[self.word_index_dict[word]] = 1 + np.log10((N+1)/(ni+1))
#        return idfs
            
    # Method: compute_idf_modified_Cosines
    # Inputs: self: 
    #               instance of LexRank
    # Outputs: idf_mod_cosines:output of idfs modified cosine similarities
    # Description:  Build an nxn matrix idf_mod_cosines where n is no of sentences
    #               coming from all the documents. 
    #               COmpute sentence similarities and store them in the 
    #               idf_mod_consines array
    #               Return the similarities
    def compute_idf_modified_cosines(self):
        n = len(self.S)
        idf_mod_cosines = np.zeros((n, n))
        idfs = self.idfs
        for i in range(n):
            tfx = self.tf_matrix[i].toarray()
            for j in range(i, n):
                tfy = self.tf_matrix[j].toarray()
                top_sum = np.dot(np.multiply(tfx, tfy), np.square(idfs))
                bottom_sum = math.sqrt(np.sum(np.square(np.multiply(tfx, idfs)))) * \
                             math.sqrt(np.sum(np.square(np.multiply(tfy, idfs))))
                if bottom_sum == 0:
                    result = 0
                else:
                    result = top_sum / bottom_sum
                idf_mod_cosines[i][j] = result
                idf_mod_cosines[j][i] = result
        return idf_mod_cosines

    # Method: compute_power_method
    # Inputs: 
    #     M: stochastic, irreducible and aperiodic matrix
    #     epsilon: error tolerance
    # Outputs: 
    #     p_curr: Eigen vector 
    # Description:  The input M is nothing but the adjacency matrix where each
    #               entry denotes whether there is an edge between 2 sentences
    #               The convergence property of Markov chains also provides us 
    #               with a simple iterative algorithm, called power method, to 
    #               compute the stationary distribution. The algorithm starts 
    #               with a uniform distribution. At each iteration, the 
    #               eigenvector is updated by multiplying with the transpose of 
    #               the stochastic matrix. Since the Markov chain is irreducible 
    #               and aperiodic, the algorithm is guaranteed to terminate. 
    #               
    def compute_power_method(self, M, epsilon):
        N = len(M)
        p_last = (1/N) * np.ones(N)
        while True:
            p_curr = M.transpose().dot(p_last)
            p_curr = p_curr / np.linalg.norm(p_curr) # Normalize or break with high nums
#             p_curr = M.dot(p_last) / np.linalg.norm(M.dot(p_last))
            delta = np.linalg.norm(p_curr - p_last)
            p_last = p_curr
            if delta < epsilon:
                break
        return p_curr
    
    # Method: compute_lexrank_score
    # Inputs: 
    #     t: cosine similarity threshold
    #     S: optional, custom list of sentences to compute score for
    # Outputs: 
    #     L:LexRank
    # Description:  If S is not passed, the instance's precomputed list of
    #               sentences is used. 
    #               The cosine similarities above threshold t are noted
    #               and the following are built
    #               CosineMatrix - adjaceny matrix which represents an edge
    #                  on a graph, where nodes are sentences
    #               Degree - No of edges/similarities that are above the threshold
    #
    #               Return Lexrank score based on the power method
    def compute_lexrank_score(self, t, S=None):
        if S is None:
            S = self.S
        n = len(S)
        CosineMatrix = np.zeros((n, n))
        Degree = np.zeros(n)
        for i in range(n):
            for j in range(n):
                if self.idf_mod_cosines[i][j] > t:
                    CosineMatrix[i][j] = 1
                    Degree[i] += 1
        for i in range(n):
            degree = Degree[i]
            for j in range(n):
                if self.idf_mod_cosines[i][j] > t:
                    CosineMatrix[i][j] /= degree
        L = self.compute_power_method(CosineMatrix, 1e-12) # epsilon can be different
        return L

    # Method: create_summary
    # Inputs: 
    #     t: cosine similarity threshold
    # Outputs: 
    #     summary: summary
    # Description:  Return summary upto length of 665 bytes
    def create_summary(self, t=0.1):
        scores = self.compute_lexrank_score(t)
        summary = ""
        prev_i = -1
        for i in np.argsort(scores)[::-1]:
            if prev_i != -1 and self.idf_mod_cosines[i][prev_i] > 0.8:
                continue

            next_sentence = self.S_orig[i]
            next_sentence = next_sentence.replace('_','')
            next_sentence = next_sentence.replace('``','')
            
            if len(summary + next_sentence) > 665:
                summary = summary[:-1]
                break
            summary += next_sentence + '\n'
            prev_i = i
        return summary

    # Method: create_summary_long_as_possible. NOT USED!!
    # Inputs: 
    #     t: cosine similarity threshold
    # Outputs: 
    #     summary: summary
    # Description:  Compute lexRank scores, sort them in descendin
    #     order and return the longest possible summary
    def create_summary_long_as_possible(self, t=0.1):
        scores = self.compute_lexrank_score(t)
        summary = ""
        prev_i = -1
        for i in np.argsort(scores)[::-1]:
            if prev_i != -1 and self.idf_mod_cosines[i][prev_i] > 0.8:
                continue           
            next_sentence = self.S_orig[i]
            if len(summary + next_sentence) <= 665:
                summary += next_sentence + '\n'
            if len(summary) >= 665 or i == np.argsort(scores)[0]:
                summary = summary[:-1]
                break
            prev_i = i
        return summary
    
    # Method: create_summary_specified_length. NOT USED!!
    # Inputs: 
    #     t: cosine similarity threshold
    # Outputs: 
    #     summary: summary
    # Description:  Compute lexRank scores, sort them in descendin
    #     order and return the summary of length 'length'  
    def create_summary_specified_length(self, length, t=0.1):
        scores = self.compute_lexrank_score(t)
        summary = ""
        for i in np.argsort(scores)[-1:-1-length:-1]:
            summary += next_sentence + '\n'
        summary = summary[:-1]
        return summary
    
    def write_summary(self, 
                      write_path='C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system', 
                      t=0.1):
        summary = self.create_summary(t)
        if not(write_path.endswith('/')):
            write_path += '/'
        f = open(write_path + self.cluster_name.upper() + '_system.txt', 'w')
        print("Writing to %s" % (write_path + self.cluster_name.upper() + '_system.txt'))
        f.write(summary)
        f.close()
		
# Example uses:
# For a task 1/2 cluster
#dir_path = './task2_docs/d30001t/'

def lexRank(threshold = 0.1):
    dir_path = "C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/task2_docs/"

    for idx,folder in enumerate(sorted(os.listdir(dir_path))):
    #    if folder != "d30031t":
    #        continue
        print("Summary for cluster "+folder)
        print("============================")
        test_lexrank = LexRank(dir_path+folder+'/', 2)
        summary = test_lexrank.create_summary(t=threshold)
        summary = summary.replace("''",'')
        print(summary)
        test_lexrank.write_summary(t=threshold)
        print("\n")
    #   debug assistance    
    #    if idx > 1: 
    #        break

# For a task 3/4 cluster
#dir_path = './ManualTranslationsFromLDC.minusHeadlines/d1003t/'
#test_lexrank = LexRank(dir_path, 4)
#summary = test_lexrank.create_summary()
#print(summary)

# To write a summary


In [2]:
lexRank(0.1)

Summary for cluster d30001t
The ruling party accused U.S. policy-makers of not taking timely action to save the Cambodian people from the Khmer Rouge.
Sam Rainsy said he had been told by Ranariddh that his party and former Khmer Rouge guerrillas had been implicated by Hun Sen in a rocket attack on the Sept. 24 opening of parliament.
After a three-month impasse, they agreed last week to a coalition deal that will make Hun Sen sole prime minister and Ranariddh president of the National Assembly.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30001T_system.txt


Summary for cluster d30002t
More than 72,000 people had been evacuated to shelters.
The hurricane has destroyed almost everything, said Mike Brown, a resident of Guanaja Island, 20 miles (32 kilometers) off the coast.
That meant the Honduran coast had been under hurricane conditions for more than a day.
Few houses have remained standing.
At 0900 GMT Tuesday, Mitch was 95 miles (152 kilometers) n

The dissident, Yao Zhenxian, who was released in April from a Chinese labor camp, is a leader of the China Democracy Party, which was formed in June during President Clinton's visit to China.
In addition, after Yao made organizational trips to several provinces, democracy activists urged him to go abroad, Wang said.
Whether he has a lawyer or not, whatever he says, the government has already decided, Hu Jiangxia said.
He, too, has no defense lawyer.
Qin, a democracy campaigner for 20 years, and Wang, a student leader of the 1989 Tiananmen Square democracy movement, are among the most active and influential members of China's dissident community.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30022T_system.txt


Summary for cluster d30024t
Even as the two leaders spoke, their angry rank-and-file lawmakers were making phone calls trying to assess whether to mount challenges against the Republican leaders who have steered the Congress since the Republic

Travel is expected to become easier once a land route between the West Bank and Gaza opens next month.
Throughout the morning, Arafat walked from the lounge to the tarmac seven times to greet planes from Egypt, Morocco, Jordan, Spain, the European Union and two Palestinian Airlines flights.
Palestine Airlines, with a small fleet that includes one Boeing 727, will begin direct flights to Jordan, Egypt and Saudi Arabia next week.
However, Israel continues to control the airspace and has the authority to shut down the airfield at any time.
Since Wye, Israel has accused the Palestinians of failing to contain anti-Israel violence, and has frozen the accords.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30040T_system.txt


Summary for cluster d30042t
The United States and Britain insist that the two suspects serve their prison term in Britain.
Louis Farrakhan, the leader of a U.S. Muslim group, met with Libyan leader Moammar Gadhafi on Sunday and congrat

Cochetel was unharmed, but he said three of the kidnappers had been killed.
Tagirov was investigating the deaths Britons Peter Kennedy, Darren Hickey, and Rudolf Petschi, and Stanley Shaw of New Zealand.
According to the Interfax news agency, Cochetel was freed early Saturday in an operation conducted by the Russian Interior Ministry and the regional security forces of the north Caucasus.
It was unclear where Cochetel was held, in the breakaway republic of Chechnya or in North Ossetia.
The victims  Peter Kennedy, Darren Hickey and Rudolf Petschi of Britain and Stanley Shaw of New Zealand  were abducted Oct. 3 by unidentified gunmen in Grozny.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30056T_system.txt


Summary for cluster d30059t
A liberal lawmaker who planned to run for president in Russia's next elections was killed Friday in St. Petersburg, a news report said.
She had said she would run again in 2000.
Starovoitova, of the reformist party Dem

Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D31033T_system.txt


Summary for cluster d31038t
Mohamed Sadeek Odeh, one of the three men in custody in the United States for the Nairobi bombing, told police in Pakistan where he was arrested after arriving from Nairobi Aug. 7 that Ahmad Khalfan was a member of the group responsible for the bombing in Dar es Salaam.
A federal district judge agreed Tuesday to review complaints by lawyers for three men arrested after the bombings of two U.S. embassies in Africa that their jail conditions in Manhattan are unconstitutional and inhumane.
Residents, who were questioned Thursday by Tanzanian police and FBI agents, told reporters Friday there were four regular visitors to the house.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D31038T_system.txt


Summary for cluster d31043t
But many legislators, who in the past gave their overwhelming support to Hariri, did not name him and

In [4]:
lexRank(0.2)

Summary for cluster d30001t
After a three-month impasse, they agreed last week to a coalition deal that will make Hun Sen sole prime minister and Ranariddh president of the National Assembly.
The deal, which will make Hun Sen prime minister and Ranariddh president of the National Assembly, ended more than three months of political deadlock that followed a July election narrowly won by Hun Sen. Key to the agreement was the formation of a Senate as the upper house of Parliament, to be led by CPP President Chea Sim, the outgoing head of the National Assembly.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30001T_system.txt


Summary for cluster d30002t
In Washington on Thursday, President Bill Clinton ordered dlrs 30 million in Defense Department equipment and services and dlrs 36 million in food, fuel and other aid be sent to Honduras, Nicaragua, El Salvador and Guatemala.
Nine other deaths had been reported elsewhere in the region by early Thursday  m

He, too, has no defense lawyer.
Wang was a student leader in the 1989 Tiananmen Square democracy demonstrations.
The dissident, Yao Zhenxian, who was released in April from a Chinese labor camp, is a leader of the China Democracy Party, which was formed in June during President Clinton's visit to China.
Wang, 32, has been imprisoned in his home city, Hangzhou in eastern China, for a month, but was not formally arrested until Nov. 30.
Qin, a democracy campaigner for 20 years, and Wang, a student leader of the 1989 Tiananmen Square democracy movement, are among the most active and influential members of China's dissident community.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30022T_system.txt


Summary for cluster d30024t
Rep. Peter King, R-N.Y., was one of several members who spoke with Livingston on Thursday.
Some moderates said they were prepared to support him.
So did McCain.
People were very angry last night, one aide said.
Even if Gingrich win

The head of the Palestinian Civil Aviation Authority, Fayez Zeidan, was not available Monday for comment.
Palestinian officials say the equipment for the control tower, the computers for the check-in counters and the flood lights are still at the Israeli port of Ashdod.
The airport has been nearly completed for months but Thursday officials there added a few extra touches of yellow paint and fresh flowers.
Zeidan was not immediately available for comment.
Every day we have differences, he told The Associated Press.
The plane will return later in the day.
Gen. Fayez Zeidan.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30040T_system.txt


Summary for cluster d30042t
But a disagreement has arisen because Libya wants the suspects jailed in the Netherlands or Libya if convicted, but the United States and Britain insist they be imprisoned in Scotland.
Libya has been under U.N. sanctions since 1992 for its refusal to hand over the two alleged intelligence

The victims  Peter Kennedy, Darren Hickey and Rudolf Petschi of Britain and Stanley Shaw of New Zealand  were abducted Oct. 3 by unidentified gunmen in Grozny.
He said the bodies had not been found.
Chechen security forces are still searching for the men's bodies.
It was not known if Kennedy spoke under duress.
Meanwhile, a Russian soldier taken hostage during the war was released after a year of efforts to free him, ITAR-Tass said Friday.
A French United Nations official who was kidnapped in southern Russia more than 10 months ago was set free Saturday and flown to U.N. headquarters in Geneva.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30056T_system.txt


Summary for cluster d30059t
A leader of the liberal Russia's Democratic Choice party, she was planning to run for president in 2000.
Two guns were found at the scene.
The report gave no other details.
A liberal lawmaker who planned to run for president in Russia's next elections was killed Frid

But more decisive leadership in the department could have assured preventive steps were taken in East Africa.
We will identify, locate and prosecute all those responsible, right up the line, from those who constructed and delivered the bombs to those who paid for them and ordered it done.
In an nearly simultaneous bombing of the U.S. Embassy in Dar es Salaam in neighboring Tanzania, 11 people were killed and 85 were injured, none of them Americans.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D31038T_system.txt


Summary for cluster d31043t
A presidential decree was expected Friday but nothing came through after a meeting between Lahoud and Hariri.
Lahoud, appointed army commander in 1989 amid a rebellion in the fractured military during the 1975-90 civil war, is credited with rebuilding the 55,000-strong Lebanese army and in ending the reign of militias.
A billionaire businessman who made his fortune in Saudi Arabia, Hariri is credited with restori

In [5]:
lexRank(0.3)

Summary for cluster d30001t
After a three-month impasse, they agreed last week to a coalition deal that will make Hun Sen sole prime minister and Ranariddh president of the National Assembly.
Hun Sen's Cambodian People's Party won 64 of the 122 parliamentary seats in July's elections, short of the two-thirds majority needed to form a government on its own.
Other details of the Senate, including how much power it will be given in the promulgation of legislation, have yet to be ironed out by the two parties.
Ranariddh and his opposition ally, Sam Rainsy, refused to accept the election results, alleging widespread intimidation and fraud by the CPP.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30001T_system.txt


Summary for cluster d30002t
The hurricane has destroyed almost everything, said Mike Brown, a resident of Guanaja Island, 20 miles (32 kilometers) off the coast.
The hurricane has destroyed almost everything, said Mike Brown, a resident of Gua

Most of the horses actually come from Europe and the United States.
If the Iranian team can borrow a couple of more horses from other participating countries, they still can compete, she said.
They are the most beautiful horses of all, she said.
The scoreless match was suspended without further play after players from both sides kicked, punched and body-slammed each other for about 10 minutes.
Officials, meanwhile, expressed shock over a brawl that erupted Tuesday during a friendly soccer match between Thailand and Qatar.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30020T_system.txt


Summary for cluster d30022t
State Department spokesman James P. Rubin said U.S. officials conveyed their concerns to the Chinese Ministry of Foreign Affairs and urged that Xu be released immediately.
Don't take this case.
Wang was a student leader in the 1989 Tiananmen Square democracy demonstrations.
Wang now faces charges that could land him in prison for five year

Hodler said he believed four agents  including one IOC member  had been involved in vote-buying over the past 10 years.
There are four agents we know, he said.
If we have to clean, we will clean, he said.
Asked whether Sydney's election for the 2000 games was clean, Hodler said, I would be surprised.
Hodler was, 3rd graf pvs
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30038T_system.txt


Summary for cluster d30040t
The head of the Palestinian Civil Aviation Authority, Fayez Zeidan, was not available Monday for comment.
The airport has been nearly completed for months but Thursday officials there added a few extra touches of yellow paint and fresh flowers.
Israeli security officials delayed two planes from taking off from the Palestinian airport on Wednesday, marking the latest tensions in a rare area of Israeli-Palestinian cooperation.
This is a preparation for the declaration of the Palestinian state, Arafat said, smiling broadly and flashing a V

Prodi's far-left ally, the Communist Refoundation Party, provoked the crisis by withdrawing its support over the weekend and saying it would not vote for his deficit-cutting 1999 budget, which is key to Italy's participation in the European common currency.
Prodi's is Italy's 55th government since World War II.
Prodi's center-left government, Italy's second-longest since World War II, was threatened with collapse by the defection of its Communist ally.
Washington maintains that NATO does not need a new U.N. resolution to order a strike.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30055T_system.txt


Summary for cluster d30056t
The victims  Peter Kennedy, Darren Hickey and Rudolf Petschi of Britain and Stanley Shaw of New Zealand  were abducted Oct. 3 by unidentified gunmen in Grozny.
Tagirov was investigating the deaths Britons Peter Kennedy, Darren Hickey, and Rudolf Petschi, and Stanley Shaw of New Zealand.
Speaking briefly, 6th graf pvs
We will

Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D31033T_system.txt


Summary for cluster d31038t
America is looking for an excuse to fire more rockets on our dear Afghanistan and that excuse is bin Laden, Saqib said.
He added, however, that the United States should know bin Laden is a good Muslim, and all the Muslim world thinks he is a good Muslim.
A nearly simultaneous attack in neighboring Kenya killed 213 people, including 12 Americans, at the U.S. Embassy in Nairobi.
At least 26 people were killed, but damage to the area was not considered extensive.
1 and blames for the Aug. 7 bombings of U.S. embassies in East Africa, Justice Noor Mohammed Saqib said an interview with The Associated Press.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D31038T_system.txt


Summary for cluster d31043t
Lahoud, 62, commander of Lebanon's army who was propelled to power with widespread popular backing, pledged in a tough inaugurati

In [6]:
lexRank(0.4)

Summary for cluster d30001t
Hun Sen's Cambodian People's Party won 64 of the 122 parliamentary seats in July's elections, short of the two-thirds majority needed to form a government on its own.
Other details of the Senate, including how much power it will be given in the promulgation of legislation, have yet to be ironed out by the two parties.
Cambodia's ruling party responded Tuesday to criticisms of its leader in the U.S. Congress with a lengthy defense of strongman Hun Sen's human rights record.
When their claims were dismissed by a Hun Sen-friendly court, they rallied their supporters into the streets of Phnom Penh.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30001T_system.txt


Summary for cluster d30002t
Nicaragua's leftist Sandinistas, who maintained close relations with Fidel Castro during their 1979-90 rule, had criticized the refusal by President Arnoldo Aleman's administration.
Eight others died in Nicaragua in flooding.
Wind-whipped 

All have spent time in prison, Xu for 12 years, much of it in solitary confinement.
Wang was a student leader in 1989's influential Tiananmen Square democracy movement.
It was the 17th international human rights treaty China has signed.
China in October signed a key U.N. treaty on civil and political rights that guarantees freedom of expression and freedom of association.
Zhu criticized the United States for condemning the arrests, saying that no country, including the United States, should interfere in China's internal affairs.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30022T_system.txt


Summary for cluster d30024t
They're going out to vote.
Another aide said, Most people are really angry with Newt, and no one knows what to do with it.
An intense struggle for control of the House is underway, with Rep. Bob Livingston conducting a telephone campaign to replace Rep. Newt Gingrich as speaker and Gingrich fighting with a counter-campaign that has 

The airport has been nearly completed for months but Thursday officials there added a few extra touches of yellow paint and fresh flowers.
Israeli security officials delayed two planes from taking off from the Palestinian airport on Wednesday, marking the latest tensions in a rare area of Israeli-Palestinian cooperation.
This is a preparation for the declaration of the Palestinian state, Arafat said, smiling broadly and flashing a V sign.
Some kissed and embraced their neighbors, others wiped away tears and several officers waved their rifles in the air.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30040T_system.txt


Summary for cluster d30042t
Sirte is 400 kilometers (250 miles) east of the Libyan capital Tripoli.
Libyan radio reported that the Parliament had been called to meet Tuesday in Sirte, 250 miles (400 kilometers) east of Tripoli, but did not say whether the Lockerbie issue would be discussed.
The aircraft was felled by a small amount of

Speaking briefly, 6th graf pvs
Their mission was to gather information to stop the spread of Islamic extremism in the region, he said in Russian.
The people that have done this to him, they deserve exactly the same back.
Everything was going right, everyone was very optimistic that the hostages would be rescued, she told Britain's ITV.
The family had expected 26-year-old Hickey to be back at his parents' pub in Thames Ditton, England, by Christmas.
He was fantastic with electrics and all that sort of stuff.
He was just making a career for himself, Hickey's sister Deborah, 21, said Wednesday, fighting back tears.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D30056T_system.txt


Summary for cluster d30059t
A liberal lawmaker who planned to run for president in Russia's next elections was killed Friday in St. Petersburg, a news report said.
The report gave no other details.
Starovoitova, 52, tried to run for president in the 1996 elections but her regi

Lahoud, 62, commander of Lebanon's army who was propelled to power with widespread popular backing, pledged in a tough inauguration speech to clean up the graft-riddled administration.
Lahoud had been expected to issue a presidential decree last week asking Hariri to form the next government after the president polled members of the 128-seat Parliament on their choice for prime minister.
He pledged in a tough inauguration speech to clean up the graft-riddled administration.
Lahoud is to be sworn in for a six-year term on Nov. 24, the day President Elias Hrawi leaves office.
Writing to C:/Users/I822363/OneDrive - SAP SE/Documents/test-summarization/system/D31043T_system.txt


Summary for cluster d31050t
Xu's daughter also criticized the verdict.
Xu is the third leading member of a would-be opposition political party put on trial for subversion in a three-week crackdown that has seen at least 32 dissidents arrested or interrogated.
1 Intermediate People's Court Monday for Xu's trial.
1 I